# Obtain GT player poses from raw poses and GT bboxes with player id

Raw poses: top-down multi-pose detection results for all people detected, with bbox conf > 0.6, keypoint conf > 0.3

GT bboxes: manually annotated player bboxes with near (1), far player ID (2) for singles. For doubles, 1,2 for near, 3,4 for doubles

Desired output: Frame, id, x1, y1, x2, y2, 0_x, 0_y... 16_x, 16_y. If pose not visible, fill with zeros. If one player is not visible, fill with 0s for bbox and pose coords

Approach: For each frame, for each GT bbox, find IoU with all multi-pose detected boxes, and grab the pose from the box with largest IoU.

In [ ]:
import cv2
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import namedtuple

In [ ]:
raw_rootdir = 'raw_pose_bbox/'
gt_rootdir = '../../../datasets/'

In [ ]:
raw_file = 'raw_pose_bbox/pose_bbox_pro/match1/1_01_00_pose_bbox.csv'
gt_file = '../../../datasets/pro/match1/bbox_id/1_01_00_gtbboxid.csv'

df_raw = pd.read_csv(raw_file)
df_gt = pd.read_csv(gt_file)

In [ ]:
df_raw.head()

,Frame,x1,y1,x2,y2,bbox_conf,0_x,0_y,1_x,1_y,...,12_x,12_y,13_x,13_y,14_x,14_y,15_x,15_y,16_x,16_y
0,0,0.471874,0.405537,0.510999,0.583816,0.993157,0.490457,0.420974,0.491763,0.416332,...,0.483928,0.495257,0.495027,0.532398,0.483928,0.530077,0.495027,0.569540,0.484581,0.567218
1,0,0.557423,0.578038,0.607258,0.854553,0.991925,0.578796,0.605582,0.579809,0.601981,...,0.587910,0.697394,0.574746,0.749600,0.585885,0.753201,0.576771,0.800006,0.580821,0.805407
2,0,0.342985,0.305662,0.377258,0.403028,0.819770,0.360656,0.326771,0.362083,0.322333,...,0.354238,0.387624,0.365826,0.413931,0.352990,0.413931,0.368679,0.413931,0.352277,0.413931
3,0,0.131730,0.315437,0.164492,0.406402,0.795672,0.146945,0.336342,0.148278,0.333973,...,0.142615,0.390826,0.153441,0.416588,0.142781,0.416588,0.156106,0.416588,0.142781,0.416588
4,1,0.471799,0.406428,0.511849,0.585271,0.995671,0.490842,0.419585,0.492152,0.416092,...,0.484292,0.495267,0.494771,0.532526,0.483637,0.530197,0.493462,0.569785,0.484947,0.566292


In [ ]:
# Note spelling error here for Visbility
df_gt.head()

,Frame,id,x1,y1,x2,y2,Visbility
0,0,1,0.551562,0.569444,0.600781,0.845833,1
1,0,2,0.460156,0.402778,0.508594,0.587500,1
2,1,1,0.559375,0.576389,0.604688,0.837500,1
3,1,2,0.471875,0.408333,0.508594,0.587500,1
4,2,1,0.557813,0.583333,0.603125,0.830556,1


## Functions

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    
    return iou

In [ ]:
def get_gt_pose(df_gt, df_raw):
    total_frame = df_gt['Frame'].to_numpy()[-1] + 1
    outrows_list = []
    for fr in range(total_frame):
        df_gt_tmp = df_gt[df_gt['Frame'] == fr]
        df_raw_tmp = df_raw[df_raw['Frame'] == fr]

        listdict_gt = df_gt_tmp.to_dict('records')
        listdict_raw = df_raw_tmp.to_dict('records')

        for i, rowi in enumerate(listdict_gt):
            outrow_dict = {}
            outrow_dict['Frame'] = fr
            outrow_dict['id'] = rowi['id']
            if rowi['Visbility'] != 0:
                bbox_gt = [rowi['x1'], rowi['y1'], rowi['x2'], rowi['y2']]
                outrow_dict['x1'], outrow_dict['y1'], outrow_dict['x2'], outrow_dict['y2'] = bbox_gt 

                iou_max = 0
                chosen_j = 0
                for j, rowj in enumerate(listdict_raw):
                    bbox_raw = [rowj['x1'], rowj['y1'], rowj['x2'], rowj['y2']]
                    iou_curr = bb_intersection_over_union(bbox_gt, bbox_raw)
                    if  iou_curr > iou_max:
                        iou_max = iou_curr 
                        chosen_j = j

                for p in range(17): # number of pose coordinates
                    if iou_max != 0: # overlapping boxes found
                        outrow_dict[str(p) + '_x'], outrow_dict[str(p) + '_y'] = listdict_raw[chosen_j][str(p) + '_x'], listdict_raw[chosen_j][str(p) + '_y']
                    else: # use pose from prev frame
                        outrow_dict[str(p) + '_x'], outrow_dict[str(p) + '_y'] = outrows_list[i-2][str(p) + '_x'], outrows_list[i-2][str(p) + '_y']
            # no player i in this frame
            else:
                outrow_dict['x1'], outrow_dict['y1'], outrow_dict['x2'], outrow_dict['y2'] = [0,0,0,0]
                for p in range(17): # number of pose coordinates
                    outrow_dict[str(p) + '_x'], outrow_dict[str(p) + '_y'] = 0, 0

            outrows_list.append(outrow_dict)

    df_out = pd.DataFrame(outrows_list)
    return df_out

In [ ]:
def save_gt_pose(raw_rootdir, gt_rootdir, out_rootdir):
    for matchdir in sorted(os.listdir(raw_rootdir)):
        outdir = os.path.join(out_rootdir, matchdir)
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        for posecsv in sorted(os.listdir(os.path.join(raw_rootdir, matchdir))):
            basename = posecsv.split('_pose_bbox.csv')[0]
            gtcsv = os.path.join(gt_rootdir, matchdir, 'bbox_id', basename + '_gtbboxid.csv')
            rawcsv = os.path.join(raw_rootdir, matchdir, posecsv)
            outcsv = os.path.join(outdir, basename + '_gtpose.csv')
            print(outcsv, ' saved')

            df_raw = pd.read_csv(rawcsv)
            df_gt = pd.read_csv(gtcsv)

            df_out = get_gt_pose(df_gt, df_raw)
            df_out.to_csv(outcsv, index=False)

## am-singles

In [ ]:
raw_rootdir_am_singles = os.path.join(raw_rootdir, 'pose_bbox_am_singles')
gt_rootdir_am_singles = os.path.join(gt_rootdir, 'am_singles')
out_rootdir_am_singles = 'gt_pose_bbox/am_singles_gtpose/'

save_gt_pose(raw_rootdir_am_singles, gt_rootdir_am_singles, out_rootdir_am_singles)

gt_pose_bbox/am_singles_gtpose/match24/1_00_01_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_01_01_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_01_02_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_01_03_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_01_04_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_02_04_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_03_04_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_03_05_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_04_05_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match24/1_05_05_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match25/1_01_00_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match25/1_02_00_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match25/1_03_00_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match25/1_04_00_gtpose.csv  saved
gt_pose_bbox/am_singles_gtpose/match25/1_05_00_gtpose.csv  saved
gt_pose_bbox/am_singles_g

## am-doubles

In [ ]:
raw_rootdir_am_doubles = os.path.join(raw_rootdir, 'pose_bbox_am_doubles')
gt_rootdir_am_doubles = os.path.join(gt_rootdir, 'am_doubles')
out_rootdir_am_doubles = 'gt_pose_bbox/am_doubles_gtpose/'

save_gt_pose(raw_rootdir_am_doubles, gt_rootdir_am_doubles, out_rootdir_am_doubles)

gt_pose_bbox/am_doubles_gtpose/match_china/doubles0_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_china/doubles1_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_china/doubles2_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_china/doubles3_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_clementi/doubles0_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_clementi/doubles1_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_clementi/doubles2_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_clementi/doubles3_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_clementi/doubles4_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_clementi/doubles5_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_clementi/doubles6_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_msia/doubles0_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_msia/doubles1_gtpose.csv  saved
gt_pose_bbox/am_doubles_gtpose/match_msia/doubles2_gtpose.csv  saved
gt

## pro

In [ ]:
raw_rootdir_pro = os.path.join(raw_rootdir, 'pose_bbox_pro')
gt_rootdir_pro = os.path.join(gt_rootdir, 'pro')
out_rootdir_pro = 'gt_pose_bbox/pro_gtpose/'

save_gt_pose(raw_rootdir_pro, gt_rootdir_pro, out_rootdir_pro)

gt_pose_bbox/pro_gtpose/match1/1_01_00_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_02_00_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_02_01_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_02_02_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_02_03_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_02_04_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_03_04_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_03_05_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_03_06_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_06_06_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_06_08_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match1/1_06_09_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match10/1_03_01_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match10/1_03_03_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match10/1_12_16_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match10/2_04_02_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/match10/2_14_08_gtpose.csv  saved
gt_pose_bbox/pro_gtpose/ma